In [ ]:
import gym
import holdem

def play_out_hand(env, n_seats):
  # reset environment, gather relevant observations
  (player_states, (community_infos, community_cards)) = env.reset()
  (player_infos, player_hands) = zip(*player_states)

  # display the table, cards and all
  env.render(mode='human')

  terminal = False
  while not terminal:
    # play safe actions, check when noone else has raised, call when raised.
    actions = holdem.safe_actions(community_infos, n_seats=n_seats)
    print('======')
    print(actions)
    print('======')
    
    (player_states, (community_infos, community_cards)), rews, terminal, info = env.step(actions)
    env.render(mode='human')

env = gym.make('TexasHoldem-v1') # holdem.TexasHoldemEnv(2)

# start with 2 players
env.add_player(0, stack=2000) # add a player to seat 0 with 2000 "chips"
env.add_player(1, stack=2000) # add another player to seat 1 with 2000 "chips"
# play out a hand
play_out_hand(env, env.n_seats)

# add one more player
# env.add_player(2, stack=2000) # add another player to seat 1 with 2000 "chips"
# play out another hand
# play_out_hand(env, env.n_seats)


In [ ]:
import holdem
env = gym.make('TexasHoldem-v1') # holdem.TexasHoldemEnv(2)
env.add_player(0, stack=2000) # add a player to seat 0 with 2000 "chips"
env.add_player(1, stack=2000)
(player_states, (community_infos, community_cards)) = env.reset()
(player_infos, player_hands) = zip(*player_states)

In [ ]:
community_infos

In [ ]:
player_states

In [ ]:
player_states

In [ ]:
player_infos

In [ ]:
player_hands

In [ ]:
rews

In [ ]:
community_cards

In [ ]:
actions = holdem.safe_actions(community_infos, n_seats=env.n_seats)
(player_states, (community_infos, community_cards)), rews, terminal, info = env.step(actions)
env.render(mode='human')

In [ ]:
community_infos

In [ ]:
community_cards

In [ ]:
[x for x in community_cards if x!=-1]

In [ ]:
import holdem.utils
hello=holdem.utils.card_to_str(16783383)
hello

In [ ]:
from treys import Card
print(Card.get_rank_int(16783383))
print(Card.get_suit_int(16783383))

In [1]:
import tensorflow as tf
class DeepQNetwork:
    def __init__(
            self,
            n_actions,
            n_features,
            learning_rate=0.01,
            reward_decay=0.9,
            e_greedy=0.9,
            replace_target_iter=300,
            memory_size=500,
            batch_size=32,
            e_greedy_increment=None,
            output_graph=False,
    ):
        self.n_actions = n_actions
        self.n_features = n_features
        self.lr = learning_rate
        self.gamma = reward_decay
        self.epsilon_max = e_greedy
        self.replace_target_iter = replace_target_iter
        self.memory_size = memory_size
        self.batch_size = batch_size
        self.epsilon_increment = e_greedy_increment
        self.epsilon = 0 if e_greedy_increment is not None else self.epsilon_max

        # total learning step
        self.learn_step_counter = 0

        # initialize zero memory [s, a, r, s_]
        self.memory = []

        # consist of [target_net, evaluate_net]
        self._build_net()
        t_params = tf.get_collection('target_net_params')
        e_params = tf.get_collection('eval_net_params')
        self.replace_target_op = [tf.assign(t, e) for t, e in zip(t_params, e_params)]

        self.sess = tf.Session()

        if output_graph:
            # $ tensorboard --logdir=logs
            # tf.train.SummaryWriter soon be deprecated, use following
            tf.summary.FileWriter("logs/", self.sess.graph)

        self.sess.run(tf.global_variables_initializer())
        self.cost_his = []

    def _build_net(self):
        # ------------------ build evaluate_net ------------------
        # input = state
        # output = action
        self.s = tf.placeholder(tf.float32, [None, self.n_features], name='s')  # input
        self.q_target = tf.placeholder(tf.float32, [None, self.n_actions], name='Q_target')  # for calculating loss
        with tf.variable_scope('eval_net'):
            # c_names(collections_names) are the collections to store variables
            c_names, n_l1, w_initializer, b_initializer = \
                ['eval_net_params', tf.GraphKeys.GLOBAL_VARIABLES], 10, \
                tf.random_normal_initializer(0., 0.3), tf.constant_initializer(0.1)  # config of layers

            # first layer. collections is used later when assign to target net
            # layer1底下的命名
            with tf.variable_scope('l1'):
                w1 = tf.get_variable('w1', [self.n_features, n_l1], initializer=w_initializer, collections=c_names)
                b1 = tf.get_variable('b1', [1, n_l1], initializer=b_initializer, collections=c_names)
                l1 = tf.nn.relu(tf.matmul(self.s, w1) + b1)

            # second layer. collections is used later when assign to target net
            # layer2底下的命名
            with tf.variable_scope('l2'):
                w2 = tf.get_variable('w2', [n_l1, self.n_actions], initializer=w_initializer, collections=c_names)
                b2 = tf.get_variable('b2', [1, self.n_actions], initializer=b_initializer, collections=c_names)
                self.q_eval = tf.matmul(l1, w2) + b2

        with tf.variable_scope('loss'):
            self.loss = tf.reduce_mean(tf.squared_difference(self.q_target, self.q_eval))
        with tf.variable_scope('train'):
            self._train_op = tf.train.RMSPropOptimizer(self.lr).minimize(self.loss)

        # ------------------ build target_net ------------------
        self.s_ = tf.placeholder(tf.float32, [None, self.n_features], name='s_')    # input
        with tf.variable_scope('target_net'):
            # c_names(collections_names) are the collections to store variables
            c_names = ['target_net_params', tf.GraphKeys.GLOBAL_VARIABLES]

            # first layer. collections is used later when assign to target net
            with tf.variable_scope('l1'):
                w1 = tf.get_variable('w1', [self.n_features, n_l1], initializer=w_initializer, collections=c_names)
                b1 = tf.get_variable('b1', [1, n_l1], initializer=b_initializer, collections=c_names)
                l1 = tf.nn.relu(tf.matmul(self.s_, w1) + b1)

            # second layer. collections is used later when assign to target net
            with tf.variable_scope('l2'):
                w2 = tf.get_variable('w2', [n_l1, self.n_actions], initializer=w_initializer, collections=c_names)
                b2 = tf.get_variable('b2', [1, self.n_actions], initializer=b_initializer, collections=c_names)
                self.q_next = tf.matmul(l1, w2) + b2

    def store_transition(self, s, a, r, s_):
        if not hasattr(self, 'memory_counter'):
            self.memory_counter = 0
        transition = (s, [a, r], s_)
        # print(transition)

        # replace the old memory with new memory
        self.memory.append(transition)
        if len(self.memory)==self.memory_size:
            del self.memory[0]


    def choose_action(self, observation):
        # to have batch dimension when feed into tf placeholder
        observation = observation[np.newaxis, :]

        if np.random.uniform() < self.epsilon:
            # forward feed the observation and get q value for every actions
            actions_value = self.sess.run(self.q_eval, feed_dict={self.s: observation})
            action = np.argmax(actions_value)
        else:
            action = np.random.randint(0, self.n_actions)
        return action

    def learn(self):
        # check to replace target parameters
        if self.learn_step_counter % self.replace_target_iter == 0:
            self.sess.run(self.replace_target_op)
            print('\ntarget_params_replaced\n')

        # sample batch memory from all memory
        if self.memory_counter > self.memory_size:
            sample_index = np.random.choice(self.memory_size, size=self.batch_size)
        else:
            sample_index = np.random.choice(self.memory_counter, size=self.batch_size)
        batch_memory = self.memory[sample_index, :]

        q_next, q_eval = self.sess.run(
            [self.q_next, self.q_eval],
            feed_dict={
                self.s_: batch_memory[:, -self.n_features:],  # fixed params
                self.s: batch_memory[:, :self.n_features],  # newest params
            })

        # change q_target w.r.t q_eval's action
        q_target = q_eval.copy()

        batch_index = np.arange(self.batch_size, dtype=np.int32)
        eval_act_index = batch_memory[:, self.n_features].astype(int)
        reward = batch_memory[:, self.n_features + 1]

        q_target[batch_index, eval_act_index] = reward + self.gamma * np.max(q_next, axis=1)

        """
        For example in this batch I have 2 samples and 3 actions:
        q_eval =
        [[1, 2, 3],
         [4, 5, 6]]
        q_target = q_eval =
        [[1, 2, 3],
         [4, 5, 6]]
        Then change q_target with the real q_target value w.r.t the q_eval's action.
        For example in:
            sample 0, I took action 0, and the max q_target value is -1;
            sample 1, I took action 2, and the max q_target value is -2:
        q_target =
        [[-1, 2, 3],
         [4, 5, -2]]
        So the (q_target - q_eval) becomes:
        [[(-1)-(1), 0, 0],
         [0, 0, (-2)-(6)]]
        We then backpropagate this error w.r.t the corresponding action to network,
        leave other action as error=0 cause we didn't choose it.
        """

        # train eval network
        _, self.cost = self.sess.run([self._train_op, self.loss],
                                     feed_dict={self.s: batch_memory[:, :self.n_features],
                                                self.q_target: q_target})
        self.cost_his.append(self.cost)

        # increasing epsilon
        self.epsilon = self.epsilon + self.epsilon_increment if self.epsilon < self.epsilon_max else self.epsilon_max
        self.learn_step_counter += 1

    def plot_cost(self):
        import matplotlib.pyplot as plt
        plt.plot(np.arange(len(self.cost_his)), self.cost_his)
        plt.ylabel('Cost')
        plt.xlabel('training steps')
        plt.show()

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:
test = np.zeros(5)
test[[0,1,2]]=1
test

In [ ]:
def transferCard(cards):
    result = []
    for i in cards:
        rank = Card.get_rank_int(i)
        suit = Card.get_suit_int(i)
        suit = int(math.log2(suit))
        result.append(suit * 13 + rank)
    one_hot_encoding = np.zeros(52)
    one_hot_encoding[result] = 1
    return one_hot_encoding
        
        
transferCard([557831, 8423187, 135427])

In [ ]:
test = np.zeros(5)
hello = [1,2,3,4,5]

np.concatenate((test,hello))

In [ ]:
test = [3, 35, 25, 2000, -1, 0]
hello = [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]
np.concatenate(())

In [ ]:
test=[]
test[0]=1

In [15]:
import math
import holdem.utils
import numpy as np
import gym
import holdem
from treys import Card

class action_table:
  CHECK = 0
  CALL = 1
  RAISE = 2
  FOLD = 3
    
def transferCard(cards):
    result = []
    for i in cards:
        rank = Card.get_rank_int(i)
        suit = Card.get_suit_int(i)
        suit = int(math.log2(suit))
        result.append(suit * 13 + rank)
    one_hot_encoding = np.zeros(52)
    one_hot_encoding[result] = 1
    return one_hot_encoding
        
def run_maze():
    step = 0
    for episode in range(300):
        # initial observation
        (player_states, (community_infos, community_cards)) = env.reset()
        (player_infos, player_hands) = zip(*player_states)
        current_player = community_infos[-1]
        totalpot = community_infos[3]
        to_call = community_infos[6]
        stack = player_infos[current_player][2]
        handrank = player_infos[current_player][4]
        betting = player_infos[current_player][6]
        cards = [x for x in community_cards if x!=-1] + player_hands[current_player] 
        cards = transferCard(cards)
        
        # print(community_infos)
        env.render(mode='human')
        terminal = False

        while not terminal:
            action = RL.choose_action(np.concatenate(([current_player, 
                                    totalpot,
                                    to_call,
                                    stack,
                                    handrank,
                                    betting],cards)))
            # first reound cannot check
            print(action == 1 and to_call == 0)
            print('action = ',action)
            print()
#             if (action == 0 and cards.sum() == 2.0):
#                 action = 1
            if(action == 1 and to_call == 0):
                action = 0
            # cannot check when encounter other raised
            elif(action == 0 and to_call != 0):
                action = 1
            elif(action == 3 and to_call == 0):
                action = 1
            # print(action, cards.sum())
            result = [[0,0],[0,0],[0,0],[0,0]]
            result[current_player][0] = action
            if action==2:
                result[current_player][1] = community_infos[5] + 10
            action = result
            # print(to_call)
            print('action = ',action)
            print('to_call = ', to_call)
            (player_states_, (community_infos_, community_cards_)), rews, terminal, info = env.step(action)
            env.render(mode='human')
            (player_infos_, player_hands_) = zip(*player_states)
            print('terminal = ', terminal)
            
            current_player_ = community_infos_[-1]
            betting_ = player_infos_[current_player][6]
            totalpot_ = community_infos_[3]
            to_call_ = community_infos_[6]
            stack_ = player_infos_[current_player][2]
            handrank_ = player_infos_[current_player][4]
            betting_ = player_infos_[current_player][6]
            cards_ = [x for x in community_cards_ if x!=-1] + player_hands[current_player_] 
            cards_ = transferCard(cards_)
            
            
            RL.store_transition(np.concatenate(([current_player, 
                                totalpot,
                                to_call,
                                stack,
                                handrank,
                                betting] , cards))
                                , action, rews, np.concatenate(([current_player_, 
                                                        totalpot_,
                                                        to_call_,
                                                        stack_,
                                                        handrank_,
                                                        betting_], cards_)))

            # swap observation
            current_player = current_player_
            betting = betting_
            to_call = to_call_
            player_states = player_states_
            community_infos = community_infos_
            community_cards = community_cards_
            
            print('========step{} finished========'.format(step))
            step += 1
        # 最開頭的200步都不更新，只管亂走存到memory
        if (step > 200):
            print('start learning')
            RL.learn()

env = gym.make('TexasHoldem-v1') # holdem.TexasHoldemEnv(2)

# start with 2 players
env.add_player(0, stack=2000) # add a player to seat 0 with 2000 "chips"
env.add_player(1, stack=2000) # add another player to seat 1 with 2000 "chips"
env.add_player(2, stack=2000)
env.add_player(3, stack=2000)
tf.reset_default_graph()

RL = DeepQNetwork(4, 58,
                  learning_rate=0.01,
                  reward_decay=0.9,
                  e_greedy=0.9,
                  replace_target_iter=200,
                  memory_size=2000,
                  output_graph=True
                  )
run_maze()
# RL.plot_cost()

total pot: 35
community:
- [  ],[  ],[  ],[  ],[  ] 
players:
0 [9♥],[5♠] stack: 2000
1 [6♣],[T♦] stack: 1990
2 [3♥],[7♠] stack: 1975
3 [A♥],[5♥] stack: 2000
False
action =  0

action =  [[0, 0], [0, 0], [0, 0], [1, 0]]
to_call =  25
total pot: 60
last action by player 3:
- call, current bet: 25
community:
- [  ],[  ],[  ],[  ],[  ] 
players:
0 [9♥],[5♠] stack: 2000
1 [6♣],[T♦] stack: 1990
2 [3♥],[7♠] stack: 1975
3 [A♥],[5♥] stack: 1975
terminal =  False
========step0 finished========
False
action =  2

action =  [[2, 60], [0, 0], [0, 0], [0, 0]]
to_call =  25
total pot: 120
last action by player 0:
^ raise, current bet: 60
community:
- [  ],[  ],[  ],[  ],[  ] 
players:
0 [9♥],[5♠] stack: 1940
1 [6♣],[T♦] stack: 1990
2 [3♥],[7♠] stack: 1975
3 [A♥],[5♥] stack: 1975
terminal =  False
========step1 finished========
False
action =  0

action =  [[0, 0], [1, 0], [0, 0], [0, 0]]
to_call =  50
total pot: 170
last action by player 1:
- call, current bet: 60
community:
- [  ],[  ],[  ],[  ],[ 

False
action =  3

action =  [[0, 0], [3, 0], [0, 0], [0, 0]]
to_call =  25
total pot: 35
last action by player 1:
x fold
community:
- [  ],[  ],[  ],[  ],[  ] 
players:
0 [T♥],[T♦] stack: 2010
1 [J♠],[4♣] stack: 1905
2 [T♠],[4♠] stack: 1895
3 [K♠],[7♠] stack: 2155
terminal =  False
========step62 finished========
False
action =  3

action =  [[0, 0], [0, 0], [3, 0], [0, 0]]
to_call =  25
total pot: 35
last action by player 2:
x fold
community:
- [  ],[  ],[  ],[  ],[  ] 
players:
0 [T♥],[T♦] stack: 2010
1 [J♠],[4♣] stack: 1905
2 [T♠],[4♠] stack: 1895
3 [K♠],[7♠] stack: 2155
terminal =  False
========step63 finished========
False
action =  2

action =  [[0, 0], [0, 0], [0, 0], [2, 60]]
to_call =  15
total pot: 95
last action by player 3:
^ raise, current bet: 60
community:
- [  ],[  ],[  ],[  ],[  ] 
players:
0 [T♥],[T♦] stack: 2010
1 [J♠],[4♣] stack: 1905
2 [T♠],[4♠] stack: 1895
3 [K♠],[7♠] stack: 2095
terminal =  False
========step64 finished========
False
action =  3

action =  [[3,

AssertionError: 

In [ ]:
test = (([0, 0, 2000, 1, -1, 0, 0, 0, 0], [33560861, 147715]), ([0, 1, 1990, 1, -1, 0, 0, 0, 0], [134228773, 8394515]), ([0, 2, 1975, 1, -1, 0, 0, 0, 0], [81922, 4228625]), ([0, 3, 2000, 1, -1, 0, 0, 0, 0], [8423187, 16787479]))
test[3]